In [4]:
from samwell.sam.sambuilder import SamBuilder

builder = SamBuilder(r1_len=50, r2_len=50)
pair1   = builder.add_pair(chrom='chr1', start1=100, start2=200)
pair2   = builder.add_pair(chrom='chr1', start1=50,  start2=250)
pair3   = builder.add_pair(chrom='chr1', start1=400, start2=500)

In [19]:
import xml.etree.ElementTree as xml
from more_itertools import flatten, only
from multipledispatch import dispatch
from toyplot.coordinates import Cartesian
from toyplot.html import RenderContext, _draw_rect, _draw_triangle
from toyplot.html import _namespace as _toyplot_namespace
from toygene.read import ToyRead, ToyReadStyle
from toygene.segment_util import to_interval
from toygene.read._toyread import ToyReadMark


def draw_read(
    parent_xml,
    length: float = 150,
    height: float = 12,
    pullback: float = 1,
    angle: float = 0.0,
) -> None:
    points = (
        (-length / 2, -height / 2),
        (-length / 2, height / 2),
        (length / 2 - pullback, height / 2),
        (length / 2, 0),
        (length / 2 - pullback, -height / 2),
    )
    points = (
        (0, 0),
        (0, height),
        (length - pullback, height),
        (length, height / 2),
        (length - pullback, 0),
    )
    xml.SubElement(
        parent_xml,
        'polygon',
        points = ' '.join([f'{x},{y}' for x, y in points]),
        attrib={'transform': f'rotate({-angle})'} if angle else {}
    )
    
def _draw_triangle(parent_xml, size, angle=0):
    markup = xml.SubElement(
        parent_xml,
        "polygon",
        points=" ".join(["%r,%r" % (xp, yp) for xp, yp in [
           (-size / 2, size / 2),
           (0, -size / 2),
           (size / 2, size / 2),
           ]]),
        )
    if angle:
        markup.set("transform", "rotate(%r)" % (-angle,))
    
        
    

class RenderToyRead(object):
    """A :class:`toygene.read.ToyRead` renderer."""

    def __init__(self, axes: Cartesian, mark: ToyReadMark, context: RenderContext) -> None:
        self.mark = mark
        self.axes = axes
        self.context = context
        self.mark_xml = xml.SubElement(
            self.context.parent,
            "g",
            id=self.context.get_id(self.mark),
            attrib={"class": "toygene-mark-ToyRead"},
        )

        self.reads_xml = xml.SubElement(self.mark_xml, "g", attrib={"class": "toyread-Reads"})
        self.mark_reads()
        
    def transform(x: int = 0, y: int = 0, angle: int = 0) -> str:
        print('hi')
        return f'translate({x},{y}) rotate({repr(-angle)})'

    def mark_reads(self) -> None:
        for i, read in enumerate(flatten(self.mark.reads)):
            one_x = float(only(self.axes.project('x', [1]))) - float(only(self.axes.project('x', [0]))) 
            one_y = float(only(self.axes.project('y', [1]))) - float(only(self.axes.project('y', [0]))) 
            x = float(only(self.axes.project('x', [0])))
            y = float(only(self.axes.project('y', [0])))
            attrib = dict(
                id=f"read-{i}-{read.query_name}",
                transform=f'translate({x},{y})' 
            )
            print(one_y, one_x)
            marker_xml = xml.SubElement(self.reads_xml, "g", attrib=attrib)
            draw_read(marker_xml, length = one_x*150, height=one_y, pullback=one_y)
            break
    
@dispatch(Cartesian, ToyReadMark, RenderContext, namespace=_toyplot_namespace)
def _render(axes: Cartesian, mark: ToyReadMark, context: RenderContext) -> None:
    RenderToyRead(axes, mark, context)

    

toyread = ToyRead([pair1, pair2, pair3])
canvas, axes, mark = toyread.draw(ToyReadStyle(height=300, width=800))

-200.0 4.635761589403977


<svg class="toyplot-canvas-Canvas" height="300.0px" id="tcc47db09714b4ae3bb029a2c1c83015d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 300.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 50 100 150 0.0 0.5 1.0

In [3]:
from toygene.read import ToyRead, ToyReadStyle

toyread = ToyRead([pair1, pair2, pair3])
canvas, axes, mark = toyread.draw(ToyReadStyle(height=200, width=500))

<svg class="toyplot-canvas-Canvas" height="200.0px" id="t415f558f36cc45e994caa0d22f400998" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 200.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 50 100 150 0.0 0.5 1.0